### TẢI CÁC THƯ VIỆN CẦN THIẾT

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
pip install WordCloud

In [ ]:
!pip install bertopic
!pip install sentence-transformers

from bertopic import BERTopic
import pandas as pd

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from sklearn.metrics.pairwise import cosine_distances
from wordcloud import WordCloud
import os


### TẢI DATASET

In [ ]:
df = pd.read_csv('data/topic_modeling_data.csv')

# BERT

### EMBEDDINGS - Tùy chỉnh

Chỉ load mô hình đã tạo, không cần chạy lại

In [ ]:
#load model
from bertopic import BERTopic
# Tải lại mô hình
topic_model = BERTopic.load("bertopic_model_after_outliers")

In [ ]:
# from bertopic import BERTopic
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=None)

# topic_model = BERTopic(
#     language="multilingual",
#     vectorizer_model=vectorizer_model, # Sử dụng N-gram ở đây
#     min_topic_size=10,
#     verbose=True
# )

# docs = df['clean_text'].tolist()
# topics, probs = topic_model.fit_transform(docs)

# topic_info = topic_model.get_topic_info()
# topic_info['Representative_Docs'] = topic_info['Representative_Docs'].apply(lambda x: [d[:100] + "..." for d in x])
# print("=== BẢNG THÔNG TIN CHỦ ĐỀ ===")
# display(topic_info)

In [ ]:
# print("Đang trích xuất embeddings từ dữ liệu")
# embeddings = topic_model._extract_embeddings(df['clean_text'].tolist(), method="document")

In [ ]:
# # Lưu ma trận embeddings vào file
# np.save('hr_embeddings.npy', embeddings)
# print("Đã lưu embeddings thành công vào file 'hr_embeddings.npy'!")

Chỉ cần load file embedding đã có sẵn

In [ ]:
print(" Đang tải embeddings từ file")

try:
    embeddings = np.load('hr_embeddings.npy')
    print(f"Kích thước ma trận: {embeddings.shape}")
except FileNotFoundError:
    print(" Không tìm thấy file! ")


### Kết quả đơn thuần

In [ ]:
current_labels = topic_model.topics_

valid_indices = [i for i, t in enumerate(current_labels) if t != -1]
valid_embeddings = embeddings[valid_indices]
valid_topics = np.array(current_labels)[valid_indices]
unique_labels = sorted(list(set(valid_topics)))

centroids = []
for label in unique_labels:
    cluster_embeddings = valid_embeddings[valid_topics == label]
    centroids.append(np.mean(cluster_embeddings, axis=0))
centroids = np.array(centroids)

intra_distances = []
for i, label in enumerate(unique_labels):
    cluster_embeddings = valid_embeddings[valid_topics == label]
    dist_to_centroid = cosine_distances(cluster_embeddings, centroids[i].reshape(1, -1))
    intra_distances.append(np.mean(dist_to_centroid))

compactness = np.mean(intra_distances)

inter_distances = cosine_distances(centroids)
separation = np.mean(inter_distances[np.triu_indices(len(unique_labels), k=1)])

print(f"Kết quả cho {len(unique_labels)} topics:")
print(f"Compactness: {compactness:.4f}")
print(f"Separation:  {separation:.4f}")

# Tune cụm

In [ ]:
def get_metrics(current_embeddings, current_topics):
    valid_idx = [i for i, t in enumerate(current_topics) if t != -1]
    if len(valid_idx) == 0: return 0, 0

    v_emb = current_embeddings[valid_idx]
    v_top = np.array(current_topics)[valid_idx]
    u_labels = sorted(list(set(v_top)))

    # 1. Centroids
    centroids = np.array([np.mean(v_emb[v_top == l], axis=0) for l in u_labels])

    # 2. Compactness
    comp_list = [np.mean(cosine_distances(v_emb[v_top == l], centroids[i].reshape(1, -1)))
                 for i, l in enumerate(u_labels)]
    avg_comp = np.mean(comp_list)

    # 3. Separation
    if len(u_labels) > 1:
        inter_dist = cosine_distances(centroids)
        avg_sep = np.mean(inter_dist[np.triu_indices(len(u_labels), k=1)])
    else:
        avg_sep = 0
    return round(avg_comp, 4), round(avg_sep, 4)


docs_list = df['clean_text'].tolist()



print(" Đang tạo bản sao lưu (Snapshot) của mô hình gốc...")

master_topic_model = copy.deepcopy(topic_model)

results = []
print(" Đang bắt đầu quá trình Tune (Sử dụng Deep Copy)...")


for n in range(31, 5, -5): 
    try:
        temp_model = copy.deepcopy(master_topic_model)

        temp_model.reduce_topics(docs_list, nr_topics=n)

        current_topics = temp_model.topics_
        actual_nr_topics = len(set(current_topics)) - (1 if -1 in current_topics else 0)

        comp, sep = get_metrics(embeddings, current_topics)
        ratio = round(sep/comp, 2) if comp != 0 else 0

        results.append({
            "Nr_Topics": actual_nr_topics,
            "Compactness": comp,
            "Separation": sep,
            "Ratio": ratio
        })

        if n % 5 == 0:
            print(f"Đã nén bản nháp về {actual_nr_topics} topics | Ratio: {ratio}")


        del temp_model

    except Exception as e:
        print(f"Lỗi ở mốc {n}: {e}")

# --- VẼ BIỂU ĐỒ ---
df_res = pd.DataFrame(results).sort_values("Nr_Topics")

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.set_xlabel('Số lượng Topic')
ax1.set_ylabel('Khoảng cách Cosine')
line1 = ax1.plot(df_res['Nr_Topics'], df_res['Compactness'], marker='o', label='Compactness (Giảm=Tốt)', color='red')
line2 = ax1.plot(df_res['Nr_Topics'], df_res['Separation'], marker='s', label='Separation (Tăng=Tốt)', color='green')

ax2 = ax1.twinx()
ax2.set_ylabel('Tỷ lệ Sep / Comp')
line3 = ax2.plot(df_res['Nr_Topics'], df_res['Ratio'], marker='^', label='Ratio (Cao=Tốt)', color='blue', linestyle='--')

lines = line1 + line2 + line3
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')

plt.title('Biểu đồ đánh giá tối ưu (Clean Run)')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

display(df_res)

# Chọn model 1 - chưa loại topic -1

In [ ]:
from bertopic import BERTopic
# Tải lại mô hình
topic_model = BERTopic.load("bertopic_model_after_outliers")

topic_model.reduce_topics(df['clean_text'].tolist(), nr_topics=6)

info_final = topic_model.get_topic_info()
display(info_final)

# Visualization model 1

In [ ]:
!pip install gensim
!pip install wordcloud

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

current_labels = np.array(topic_model.topics_)
unique_labels = sorted([l for l in set(current_labels) if l != -1])

# Lấy bảng thông tin topic hiện tại để tra cứu tên
df_info = topic_model.get_topic_info()

topic_metrics = []

for label in unique_labels:
    # Lấy các vector thuộc cụm này
    idx = np.where(current_labels == label)[0]
    cluster_embeddings = embeddings[idx]

    # Tính Tâm cụm
    centroid = np.mean(cluster_embeddings, axis=0).reshape(1, -1)

    # 1. Compactness: Khoảng cách TB từ các điểm đến tâm
    dist_to_centroid = cosine_distances(cluster_embeddings, centroid)
    comp_i = np.mean(dist_to_centroid)

    # 2. Lấy tên Topic an toàn
    name_row = df_info[df_info['Topic'] == label]
    topic_name = name_row['Name'].values[0] if not name_row.empty else f"Topic {label}"

    topic_metrics.append({
        "Topic_ID": label,
        "Topic_Name": topic_name,
        "Compactness": round(comp_i, 4),
        "Size": len(idx)
    })

all_centroids = np.array([np.mean(embeddings[current_labels == l], axis=0) for l in unique_labels])
if len(unique_labels) > 1:
    inter_dist_matrix = cosine_distances(all_centroids)
    # Tính Separation cho từng topic
    for i, m in enumerate(topic_metrics):
        sep_i = np.mean([dist for j, dist in enumerate(inter_dist_matrix[i]) if i != j])
        m["Separation"] = round(sep_i, 4)
else:
    for m in topic_metrics: m["Separation"] = 0

# Hiển thị kết quả
df_final_metrics = pd.DataFrame(topic_metrics)
display(df_final_metrics)

print(f"\nTrung bình Compactness: {df_final_metrics['Compactness'].mean():.4f}")
print(f"Trung bình Separation:  {df_final_metrics['Separation'].mean():.4f}")

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)

In [ ]:
all_topics = sorted([t for t in set(topic_model.topics_)])

display_topics = all_topics[:10]

print(f"Đang hiển thị WordCloud cho {len(display_topics)} chủ đề...")

for topic_id in display_topics:
    # 1. Lấy dữ liệu từ model
    topic_data = topic_model.get_topic(topic_id)
    if not topic_data: continue # Bỏ qua nếu lỗi

    words_dict = {word: prob for word, prob in topic_data}

    # 2. Tạo WordCloud
    wc = WordCloud(
        background_color="white",
        width=1000,          # Tăng kích thước ảnh cho nét
        height=600,
        max_words=50,        # Số lượng từ hiển thị
        colormap='viridis'   # Màu sắc
    )
    wc.generate_from_frequencies(words_dict)

    # 3. Lấy tên Topic làm tiêu đề
    info = topic_model.get_topic_info(topic_id)
    topic_name = info['Name'].values[0]
    clean_name = " ".join(topic_name.split("_")[1:])

    # 4. Vẽ hình riêng biệt
    plt.figure(figsize=(10, 6)) # Tạo khung hình mới cho mỗi vòng lặp
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Topic {topic_id}: {clean_name}", fontsize=16, fontweight='bold', color='#2c3e50')
    plt.show()

    print("-" * 50)

# Model 2 - Reduce OUTLIERS nhưng chỉ số giảm

In [ ]:
new_topics = topic_model.reduce_outliers(df['clean_text'].tolist(), topic_model.topics_, strategy="c-tf-idf")

# Cập nhật lại model với nhãn mới
topic_model.update_topics(df['clean_text'].tolist(), topics=new_topics)
info = topic_model.get_topic_info()
display(info)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances

current_labels = np.array(topic_model.topics_)
unique_labels = sorted([l for l in set(current_labels) if l != -1])

df_info = topic_model.get_topic_info()

topic_metrics = []


for label in unique_labels:
    # Lấy các vector thuộc cụm này
    idx = np.where(current_labels == label)[0]
    cluster_embeddings = embeddings[idx]

    # Tính Tâm cụm
    centroid = np.mean(cluster_embeddings, axis=0).reshape(1, -1)

    # 1. Compactness: Khoảng cách TB từ các điểm đến tâm
    dist_to_centroid = cosine_distances(cluster_embeddings, centroid)
    comp_i = np.mean(dist_to_centroid)

    # 2. Lấy tên Topic an toàn
    name_row = df_info[df_info['Topic'] == label]
    topic_name = name_row['Name'].values[0] if not name_row.empty else f"Topic {label}"

    topic_metrics.append({
        "Topic_ID": label,
        "Topic_Name": topic_name,
        "Compactness": round(comp_i, 4),
        "Size": len(idx)
    })


all_centroids = np.array([np.mean(embeddings[current_labels == l], axis=0) for l in unique_labels])
if len(unique_labels) > 1:
    inter_dist_matrix = cosine_distances(all_centroids)
    # Tính Separation cho từng topic
    for i, m in enumerate(topic_metrics):
        sep_i = np.mean([dist for j, dist in enumerate(inter_dist_matrix[i]) if i != j])
        m["Separation"] = round(sep_i, 4)
else:
    for m in topic_metrics: m["Separation"] = 0

# Hiển thị kết quả
df_final_metrics = pd.DataFrame(topic_metrics)
display(df_final_metrics)

print(f"\nTrung bình Compactness: {df_final_metrics['Compactness'].mean():.4f}")
print(f"Trung bình Separation:  {df_final_metrics['Separation'].mean():.4f}")

# Visualize Model 2

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os

all_topics = sorted([t for t in set(topic_model.topics_)])

# Giới hạn số lượng nếu quá nhiều (Ví dụ chỉ hiện 10 topic đầu)
# Bỏ dòng này nếu muốn hiện hết
display_topics = all_topics[:10]

print(f"Đang hiển thị WordCloud cho {len(display_topics)} chủ đề...")

for topic_id in display_topics:
    # 1. Lấy dữ liệu từ model
    topic_data = topic_model.get_topic(topic_id)
    if not topic_data: continue # Bỏ qua nếu lỗi

    words_dict = {word: prob for word, prob in topic_data}

    # 2. Tạo WordCloud
    wc = WordCloud(
        background_color="white",
        width=1000, 
        height=600,
        max_words=50,
        colormap='viridis'
    )
    wc.generate_from_frequencies(words_dict)

    # 3. Lấy tên Topic làm tiêu đề
    info = topic_model.get_topic_info(topic_id)
    topic_name = info['Name'].values[0]
    clean_name = " ".join(topic_name.split("_")[1:])

    # 4. Vẽ hình riêng biệt
    plt.figure(figsize=(10, 6)) # Tạo khung hình mới cho mỗi vòng lặp
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Topic {topic_id}: {clean_name}", fontsize=16, fontweight='bold', color='#2c3e50')
    plt.show()

    print("-" * 50)